[![Colabで開く](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/schwalbe1996/ds_media_intro/blob/main/chap13.ipynb)

# 13章「画像のフィルタ処理」

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

image = cv2.imread('sample.png',cv2.IMREAD_GRAYSCALE)

# 平均0，分散15のガウス分布に従うノイズを用意する．
sigma = 15
noise = np.random.normal(0, sigma, image.shape)

src = image + noise
# ノイズ加えたあとの画素値が0～255の範囲に収まるように調整
src[src > 255] = 255
src[src < 0] = 0
src = src.astype(np.uint8)

plt.figure(figsize=(15,3))
plt.subplot(1,3,1)
plt.imshow(image, cmap='gray')
plt.subplot(1,3,2)
plt.imshow(noise, cmap='gray')
plt.colorbar()
plt.subplot(1,3,3)
plt.imshow(src,cmap='gray')
plt.show()

In [ ]:
x = np.arange(0, 7, 0.1)
y = np.sin(x)

sigma = 0.1
noise = np.random.normal(0, sigma, y.shape)

y2 = y + noise

mean = np.convolve(y2, np.ones(5)/5, mode='same')

#plt.plot(x,y2,linestyle='solid',label='noisy',color='black')
#plt.plot(x,mean,linestyle='dashed',label='mean',color='black')
plt.plot(x,y2,linestyle='solid',label='noisy')
plt.plot(x,mean,linestyle='dashed',label='mean')
plt.legend()
plt.show()

In [ ]:
N = 5
K = np.ones((N,N))/N**2
result = cv2.filter2D(src, ddepth=-1, kernel=K)
plt.imshow(result,cmap='gray')
plt.show()

In [ ]:
N = 5
sigma = 0 # σを0に設定するとNの値から自動的に適切なものを計算する
result = cv2.GaussianBlur(src, (N,N), sigma)
# カーネルKを計算してから畳み込みを行う場合
K = cv2.getGaussianKernel(ksize=N, sigma=sigma)
result = cv2.filter2D(src, ddepth=-1, kernel=K)
plt.imshow(result,cmap='gray')
plt.show()

In [ ]:
N = 5
result = cv2.medianBlur(image, N)
plt.imshow(result,cmap='gray')
plt.show()

In [ ]:
N = 15
sigma_c = 30
sigma_s = 20
result = cv2.bilateralFilter(src, N, sigma_c, sigma_s)
plt.imshow(result,cmap='gray')
plt.show()

In [ ]:
K_x = np.array([[0,0,0],[-1,0,1],[0,0,0]])
# (注)微分の結果が負になる場合があるので，ddepthを浮動小数点型にする
diff_x = cv2.filter2D(image, ddepth=cv2.CV_64F, kernel=K_x)


K_y = np.array([[0,-1,0],[0,0,0],[0,1,0]])
diff_y = cv2.filter2D(image, ddepth=cv2.CV_64F, kernel=K_y)

plt.imshow(diff_x,cmap='gray')
plt.colorbar()
plt.show()
plt.imshow(diff_y,cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
K_laplacian = np.array([[0,1,0],[1,-4,1],[0,1,0]])
result_l = cv2.filter2D(image, ddepth=cv2.CV_64F, kernel=K_laplacian)

K_sharp = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])
result_s = cv2.filter2D(image, ddepth=cv2.CV_64F, kernel=K_sharp)
# 0～255の範囲から外れている画素があるので調整
result_s[ result_s < 0 ] = 0
result_s[ result_s > 255] = 255

plt.imshow(image,cmap='gray')
plt.show()
plt.imshow(result_l,cmap='gray')
plt.colorbar()
plt.show()
plt.imshow(result_s,cmap='gray')
plt.show()

In [ ]:
freq = np.fft.fft2(image)
print(freq.dtype) # フーリエ変換の結果は複素数(complex128)になる

In [ ]:
def show_image3d(data,elev=30, azim=-75):
    x = np.arange(data.shape[1])
    y = np.arange(data.shape[0])
    xx,yy = np.meshgrid(x,y)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=elev, azim=azim) # 3次元の視点角を設定します。

    sp = ax.plot_surface(xx,yy,data,cmap='jet',vmin=None,vmax=None)
    fig.colorbar(sp,shrink=0.5)
    ax.invert_yaxis()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    fig.show()

In [ ]:
# 離散フーリエ変換の結果は複素数になるので，可視化のために対数パワースペクトルに変換
power_spectrum = np.log( np.abs(freq) )
# 上の結果だと低周波成分が図中の端にきて見にくいので，低周波成分が図の中央に寄るように加工
power_spectrum = np.fft.fftshift(power_spectrum)
plt.imshow(power_spectrum)
plt.colorbar()
plt.show()

In [ ]:
show_image3d(power_spectrum)

In [ ]:
# （2次元）離散逆フーリエ変換
rev_image = np.fft.ifft2(freq)
# 逆フーリエ変換の結果も複素数になるので，ここでは絶対値を計算
rev_image = np.abs(rev_image)
plt.imshow(rev_image,cmap='gray')
plt.show()

In [ ]:
filter = np.zeros(image.shape)
filter[-1,-1] = 1; filter[-1,0] = 1; filter[-1,1] = 1; 
filter[0,-1] = 1; filter[0,0] = 1; filter[0,1] = 1; 
filter[1,-1] = 1; filter[1,0] = 1; filter[1,1] = 1;
filter /= 9

freq_filter =np.fft.fft2(filter)
power_spectrum = np.abs(freq_filter)
power_spectrum = np.fft.fftshift(power_spectrum)
plt.imshow(power_spectrum,cmap='gray')
plt.colorbar()
plt.show()
show_image3d(power_spectrum) # 3次元表示したい場合はこちら

In [ ]:
filter = np.zeros(image.shape)
filter[-1,0] = -1
filter[0,-1] = -1; filter[0,0] = 5; filter[0,1] = -1; 
filter[1,0] = -1

freq_filter =np.fft.fft2(filter)
power_spectrum = np.abs(freq_filter)
power_spectrum = np.fft.fftshift(power_spectrum)
plt.imshow(power_spectrum,cmap='gray')
plt.colorbar()
plt.show()
show_image3d(power_spectrum) # 3次元表示したい場合はこちら

In [ ]:
freq = np.fft.fft2(image) # 画像を離散フーリエ変換
            
s = 50
freq[120-s:120+s,160-s:160+s] = 0 # 高周波成分を0にする
rev_image = np.fft.ifft2(freq) # 逆フーリエ変換
rev_image = np.abs(rev_image)
plt.imshow(rev_image, cmap='gray')
plt.show()    